# Extended conversations while following up the topic

You might have noticed, the openAI ChatCompletion api takes an array of messages. So far we've been passing only 1 message (the prompt) with role: "user".
It can be a sequence of messages of a conversation, so that the model can have a context of previous discussions, and continue the

This notebook explores how we can create personalized or specialized ChatBots for specific tasks or behaviors.

## Setup
Import necessary libraries and load openAI API key

In [ ]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv("OPENAI_API_KEY")

#### Helper functions to get GPT response

In [ ]:
# Old function, that takes 1 msg as prompt and responds. It doesn't have any record of previous conversations
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=temperature, # this is the degree of randomness of the model's output
  )
  return response.choices[0].message["content"]

# New function takes a messages array
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=temperature, # this is the degree of randomness of the model's output
  )
  # print(str(response.choices[0].message))
  return response.choices[0].message["content"]

## Roles for messages
- system: What kind of bot it is, A personality
- assistant: Responder from the system, carrying the system's personality
- user: User who is interacting with the system

In [ ]:
messages =  [
  # Defining the personality of the Chat-Bot
  { 'role':'system', 'content':'You are an assistant that speaks like Shakespeare.' },
  # Conversation so far...
  { 'role':'user', 'content':'tell me a joke' },
  { 'role':'assistant', 'content':'Why did the chicken cross the road' },
  { 'role':'user', 'content':'I don\'t know' }
]

# bot will continue the conversation with an answer with the role: assistant
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
  { 'role':'system', 'content':'You are friendly chatbot.' },
  { 'role':'user', 'content':'Hi, my name is Isa' }
]

# friendly chatbot will respond with a greeting
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
  { 'role':'system', 'content':'You are friendly chatbot.' },
  { 'role':'user', 'content':'Yes, can you remind me, What is my name?' }
]

# Here the friendly chat-bot doesn't have any context for the given prompt.
# Of course it doesn't know the user's name, we never told it. It's going to respond with an error
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
  { 'role':'system', 'content':'You are friendly chatbot.' },
  { 'role':'user', 'content':'Hi, my name is Isa' },
  { 'role':'assistant', 'content': "Hi Isa! It's nice to meet you. Is there anything I can help you with today?" },
  { 'role':'user', 'content':'Yes, you can remind me, What is my name?' }
]

# This time the bot has context, the user has provided their name previously. The bot can answer
response = get_completion_from_messages(messages, temperature=1)
print(response)

## Order Bot


#### Configure the chatbot Role

In [ ]:
system_persona = """
You are OrderBot, an automated service to collect orders for a pizza restaurant.
You first greet the customer, then collects the order, and then asks if it's a
pickup or delivery. You wait to collect the entire order, then summarize it 
and check for a final time if the customer wants to add anything else. 
If it's a delivery, you ask for an address. Finally you collect the payment.
Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.
You respond in a short, very conversational friendly style.

The menu includes 
pepperoni pizza  12.95, 10.00, 7.00
cheese pizza   10.95, 9.25, 6.50
eggplant pizza   11.95, 9.75, 6.75
fries 4.50, 3.50
greek salad 7.25

Toppings:
extra cheese 2.00,
mushrooms 1.50
sausage 3.00
canadian bacon 3.50
AI sauce 1.50
peppers 1.00

Drinks:
coke 3.00, 2.00, 1.00
sprite 3.00, 2.00, 1.00
bottled water 5.00
"""

context = [{ 'role':'system', 'content': system_persona }]  # We'll accumulate messages in this context

#### Setup Chat-UI for placing order

In [ ]:
import panel as pn # GUI

pn.extension()
# input box for the user to type msg
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
# Send button
button_conversation = pn.widgets.Button(name="Chat!")

# We'll add assistant response and user msgs of the conversation between them.
panels = [] # collect display 

def collect_messages(_):
  # collect the user msg
  prompt = inp.value_input
  # reset input box
  inp.value = ''
  # Append user msg
  context.append({ 'role':'user', 'content':f"{prompt}" })
  # get response
  response = get_completion_from_messages(context) 
  # append bot response
  context.append({ 'role':'assistant', 'content':f"{response}" })
  # Append the msgs to UI
  panels.append(
    pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
  panels.append(
    pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

  return pn.Column(*panels)

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
  inp,
  pn.Row(button_conversation),
  pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

# Render the Chat UI with msgs
dashboard

#### Generate JSON object for the order details

In [ ]:
messages =  context.copy()
messages.append(
{
  'role':'system', 
  'content':"""
  create a json summary of the previous food order. 
  Itemize the price for each item The fields should be 
    1) pizza, include size 
    2) list of toppings 
    3) list of drinks, include size   
    4) list of sides include size  
    5)total price
    """
})

# The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages)
print(response)